# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Шаги по выполнению проекта**

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import re 
from tqdm import tqdm
import numpy as np 
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier
from nltk.corpus import wordnet
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head(-5) 

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159282,159441,"""\nNo he did not, read it again (I would have ...",0
159283,159442,"""\n Auto guides and the motoring press are not...",0
159284,159443,"""\nplease identify what part of BLP applies be...",0
159285,159444,Catalan independentism is the social movement ...,0


Сначала можно подумать что 'Unnamed' дублирует индексы, но по последним строкам видно что они не совпадают. Наверно это зашифрованные никнеймы

In [3]:
df['toxic'].mean() # мы видим что токсичных комментариев около 10%, серьёзный дисбаланс при исследовании данных

0.10161213369158527

In [4]:
df.isna().sum() # пропусков нет

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [5]:
corpus = df['text'] # cоздадим корпус постов

In [6]:
def clear_text(text): # создадим функцию для очистки текста от лишних символов
    clear_text = re.sub(r'[^a-zA-Z ]', ' ',text)
    return " ".join(clear_text.split())

In [7]:
clear_text(corpus[0]) # тестируем работу, все хорошо

'Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalisms just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now'

In [8]:
df['lemma_text'] = df['text'].apply(clear_text)
df.head(5)

,Unnamed: 0,text,toxic,lemma_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [9]:
def get_wordnet_pos(text): # создадим функцию для применения POS тегов для преобразования слов в начальные формы
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([text])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [10]:
def lemmatize(text):  # создадим функцию лемматизации
    lemmatizer = WordNetLemmatizer() # объявим лемматизатор
    lemm_list = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)] # применим лемматизацию к ранее очищенному тексту
    lemm_text = " ".join(lemm_list) # добавим пробелы
    
    return lemm_text

In [11]:
df['lemma_text'] = df['lemma_text'].apply(lemmatize) # изменим ранее созданную колонку
df.head(5) # теперь у нас есть колонка с исходным текстом и лемматизированным и очищенным

,Unnamed: 0,text,toxic,lemma_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not try to edit war It s ju...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


In [12]:
features = df['lemma_text']
target = df['toxic']

In [13]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345, stratify=target) # выделим тестовыую и обучающую выборки
print(features_train.shape)
features_test.shape

(119469,)


(39823,)

In [14]:
stopwords = set(nltk_stopwords.words('english')) # опеределим набор стоп слов
count_tf_idf = TfidfVectorizer(stop_words=stopwords) # опеределим векторизатор

### Промежуточный вывод:

Мы проанализировали наш файл и увидели что доля ответов с классификацией 1 составляет примерно 10%.

Мы создали корпус из текста, очистили его и лемматизировали. Затем мы его так же убрали от ненужных слов с помощью стоп-слов и векторизовали признаки для того чтобы передать их для машинного обучения.

## Обучение

### Логистическая регрессия

In [15]:
regression = Pipeline([    # построим пайплайн для модели и векторизации
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('model', LogisticRegression(class_weight='balanced', solver='liblinear', random_state=12345) )
])

In [16]:
grid_space={
    'model__C': [0.1, 10.0, 1.0],
           }

grid = GridSearchCV(regression, scoring="f1", param_grid=grid_space , cv=3, n_jobs=-1)

grid.fit(features_train, target_train)

print('Лучшие гиперпараметры: ', grid.best_params_)
print('Лучший результат f1_score: ', grid.best_score_)

Лучшие гиперпараметры:  {'model__C': 10.0}
Лучший результат f1_score:  0.7588922668943111


Мы получили результат 0.76 и для f1 меры логистической регрессии для кросс-валидации и лучший параметр С = 10

### Случайный лес

In [17]:
forest = Pipeline([    
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('model', RandomForestClassifier(random_state=12345, class_weight='balanced') )
])

In [18]:
grid_space={
    'model__max_depth':[x for x in range(3,7,1)], 
    'model__n_estimators':[x for x in range(10,31,10)]
}

In [19]:
grid = GridSearchCV(forest, scoring="f1", param_grid=grid_space , cv=3, n_jobs=-1)

grid.fit(features_train, target_train)

print('Лучшие гиперпараметры: ', grid.best_params_)
print('Лучший результат f1_score: ', grid.best_score_)

Лучшие гиперпараметры:  {'model__max_depth': 6, 'model__n_estimators': 30}
Лучший результат f1_score:  0.30090376205370695


Случайный лес работает долго и результат меры f1 составляет 0.30 что существенно хуже регрессии

### CatBoost

In [20]:
cbclass = Pipeline([    
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('model', CatBoostClassifier(verbose=0))
])

In [21]:
grid_space={'model__depth'         : [3,4,5],
            'model__iterations'    : [10]
}

grid = GridSearchCV(cbclass, scoring='f1', param_grid=grid_space , cv=3, n_jobs=-1)

grid.fit(features_train, target_train)

print('Лучшие гиперпараметры: ', grid.best_params_)
print('Результат f1_score: ', grid.best_score_)

Лучшие гиперпараметры:  {'model__depth': 5, 'model__iterations': 10}
Результат f1_score:  0.5899064003600368


Результат CatBoost f1-меры на кросс-валидационной выборке составляет 0.59

### Промежуточный вывод:

Линейная модель показала себя лучше всего с результатом 0.76. Вероятно это связано с тем что модели основанный на принятии решения по дереву склонны к переобучению. 

### Логистическая регрессия на тестовой выборке

In [22]:
features_train_vector = count_tf_idf.fit_transform(features_train)
features_test_vector = count_tf_idf.transform(features_test) 

In [23]:
model_lr = LogisticRegression(C=10.0, class_weight='balanced', solver='liblinear', random_state=12345).fit(features_train_vector, target_train)

predictions = model_lr.predict(features_test_vector)

print('Результат f1-core на тестовой выборке:', f1_score(target_test,predictions))

Результат f1-core на тестовой выборке: 0.7587633605332721


Мы получили результат 0.77 на тестовой выборке и выполнили задачу по результату.

Проверим нашу модель на адекватность с помощью Дамми классификатора:

In [24]:
dummy_regr = DummyClassifier(strategy="uniform")
dummy_regr.fit(features_train_vector, target_train)
DummyClassifier()
dummy_regr_predict = dummy_regr.predict(features_test_vector)
f1 = f1_score(target_test,dummy_regr_predict)
print('Результат f1_score для модели Дамми составляет:', f1)

Результат f1_score для модели Дамми составляет: 0.17086308405361392


Результат Дамми показал 0.17 на тестовой выборке, что существенно хуже нашего результата и может говорить об адекватности нашей модели.

## Выводы

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Мы обучили модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Была задача получить на модели значение метрики качества F1 не меньше 0.75.

Мы проанализировали наш файл и увидели что доля ответов с классификацией 1 составляет примерно 10%.

Мы создали корпус из текста, очистили его и лемматизировали. Затем мы его так же убрали от ненужных слов с помощью стоп-слов и векторизовали признаки для того чтобы передать их для машинного обучения.

Затем мы приступили к обучению.

Модель линейной регрессии показала результат 0.76 на тестовой выборке что полностью устраивает нас по нашей задаче. 

Случайный лес работает долго и результат меры f1 составляет 0.30 что существенно хуже регрессии

CatBoost показал результат в 0.59

Результат Дамми показал 0.17 на тестовой выборке, что существенно хуже нашего результата и может говорить об адекватности нашей модели.